In [10]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

In [11]:
##---------------DATOS DE ENTRENAMIENTO -------------------------##
##Cargar los datos con extención .mat
x = loadmat('/DatosEntrenamiento2.mat')

##Traernos los datos del archivo .mat que nos interesan
y = x['data_te']

##Convertir los datos en un array de Numpy
z = np.array(y)

##Convertir el Array en un DataFrame de Pandas
df = pd.DataFrame(z)

##Contar el numero de datos nulos del dataFrame
np.count_nonzero(pd.isnull(df) == True)

df.columns = ['uplink_size_mean','uplink_size_median','uplink_size_MAD','uplink_size_STD','uplink_size_Skewness','uplink_size_Kurtosis','uplink_size_MAX','uplink_size_MIN','uplink_size_MeanSquare','downlink_size_mean','downlink_size_median','downlink_size_MAD', 'downlink_size_STD','downlink_size_Skewness','downlink_size_Kurtosis','downlink_size_MAX','downlink_size_MIN','downlink_size_MeanSquare','both_links_size_mean','both_links_size_mean','both_links_size_MAD','both_links_size_STD','both_links_size_Skewness','both_links_size_Kurtosis','both_links_size_MAX','both_links_size_MIN','both_links_size_MeanSquare','uplink_interval_mean','uplink_interval_median','uplink_interval_MAD','uplink_interval_STD','uplink_interval_Skewness','uplink_interval_Kurtosis','uplink_interval_MAX','uplink_interval_MIN','uplink_interval_MeanSquare','downlink_interval_mean','downlink_interval_median','downlink_interval_MAD','downlink_interval_STD','downlink_interval_Skewness','downlink_interval_Kurtosis','downlink_interval_MAX','downlink_interval_MIN','downlink_interval_MeanSquare','both_links_interval_mean','both_links_interval_median','both_links_interval_MAD','both_links_interval_STD','both_links_interval_Skewness','both_links_interval_Kurtosis','both_links_interval_MAX','both_links_interval_MIN','both_links_interval_MeanSquare','Label']
df.sample(8)

,uplink_size_mean,uplink_size_median,uplink_size_MAD,uplink_size_STD,uplink_size_Skewness,uplink_size_Kurtosis,uplink_size_MAX,uplink_size_MIN,uplink_size_MeanSquare,downlink_size_mean,downlink_size_median,downlink_size_MAD,downlink_size_STD,downlink_size_Skewness,downlink_size_Kurtosis,downlink_size_MAX,downlink_size_MIN,downlink_size_MeanSquare,both_links_size_mean,both_links_size_mean,both_links_size_MAD,both_links_size_STD,both_links_size_Skewness,both_links_size_Kurtosis,both_links_size_MAX,both_links_size_MIN,both_links_size_MeanSquare,uplink_interval_mean,uplink_interval_median,uplink_interval_MAD,uplink_interval_STD,uplink_interval_Skewness,uplink_interval_Kurtosis,uplink_interval_MAX,uplink_interval_MIN,uplink_interval_MeanSquare,downlink_interval_mean,downlink_interval_median,downlink_interval_MAD,downlink_interval_STD,downlink_interval_Skewness,downlink_interval_Kurtosis,downlink_interval_MAX,downlink_interval_MIN,downlink_interval_MeanSquare,both_links_interval_mean,both_links_interval_median,both_links_interval_MAD,both_links_interval_STD,both_links_interval_Skewness,both_links_interval_Kurtosis,both_links_interval_MAX,both_links_interval_MIN,both_links_interval_MeanSquare,Label
6629,0.005218,0.007738,0.000641,0.000931,1.255973,2.998328,0.022081,5.870000e-06,0.009204,115.200000,57.985630,88.0,17.7912,1.226663,2.601422,224.0,76.0,128.447966,0.001745,0.005276,0.000190,0.000266,3.506527,14.244762,0.026233,0.000001,0.005523,990.480000,709.190676,1259.0,520.3926,-0.442565,1.306181,1610.0,76.0,1215.440672,0.001309,0.004260,0.000127,0.000178,3.889538,17.336670,0.021907,1.254000e-06,0.004436,771.66,722.396114,224.0,219.4248,0.172056,1.111598,1610.0,76.0,1054.559975,1.0
9737,0.013120,0.013708,0.006874,0.009025,0.976309,-0.074978,0.051075,1.760000e-04,0.018809,661.300000,627.356641,232.0,5.9304,0.833387,-1.211990,1676.0,228.0,904.309036,0.005647,0.011531,0.000382,0.000433,2.308222,4.270250,0.046493,0.000058,0.012765,299.657143,283.782979,228.0,0.0000,3.754864,12.349237,1561.0,220.0,411.310970,0.004120,0.007905,0.000562,0.000667,2.345413,4.691613,0.038365,5.800000e-05,0.008879,408.15,446.274961,228.0,0.0000,2.186860,3.009182,1676.0,220.0,603.122003,0.0
9571,7.328514,41.591303,0.005554,0.007143,5.232612,26.177193,239.008522,1.500000e-04,41.606782,274.606061,59.073545,240.0,17.7912,1.400658,0.880595,439.0,228.0,280.699883,5.399361,31.439572,0.003249,0.004642,6.524190,43.712187,238.999006,0.000102,31.667757,257.283582,49.041975,228.0,17.7912,1.436742,1.155198,411.0,204.0,261.847398,3.617638,25.784649,0.001359,0.001741,8.121745,68.371174,238.999006,1.020000e-04,25.909206,263.00,52.899944,240.0,17.7912,1.492057,1.437060,439.0,204.0,268.215249,0.0
6794,0.005686,0.008232,0.000471,0.000668,1.090654,2.450051,0.022073,2.151000e-06,0.009908,123.657143,63.035911,88.0,17.7912,0.818824,1.751024,224.0,76.0,138.387448,0.002853,0.005688,0.000257,0.000351,1.999564,5.590649,0.022231,0.000002,0.006324,844.738462,752.989132,709.0,938.4858,0.005308,1.026553,1610.0,76.0,1127.764487,0.001990,0.004768,0.000112,0.000153,2.611680,8.749995,0.022064,2.151000e-06,0.005144,592.36,698.133725,88.0,17.7912,0.742199,1.586115,1610.0,76.0,912.911365,1.0
11959,0.003005,0.004004,0.001589,0.002113,1.781828,3.000428,0.017626,1.010000e-04,0.004963,354.216200,323.709000,228.0,0.0000,2.048589,2.259971,1162.0,228.0,476.890500,0.001765,0.002556,0.000395,0.000239,2.116970,3.989856,0.011561,0.000107,0.003089,1521.032000,358.283500,1646.0,0.0000,-2.758640,6.315540,1646.0,228.0,1562.007000,0.001113,0.002031,0.000365,0.000259,3.162543,10.240620,0.011342,1.010000e-04,0.002308,1089.31,662.614200,1646.0,0.0000,-0.448942,-1.725560,1646.0,228.0,1273.288000,0.0
4735,0.015004,0.031919,0.001986,0.002944,2.769560,9.271236,0.117752,8.160000e-07,0.034687,126.080000,41.111556,88.0,2.9652,0.079239,1.007393,168.0,86.0,132.358302,0.004752,0.020448,0.000029,0.000041,5.452718,31.983165,0.124526,0.000002,0.020860,962.413333,616.936286,1334.0,237.2160,-0.659331,1.545447,1574.0,76.0,1140.953549

In [12]:
YEntrenamiento = df['Label']
XEntrenamiento = scaler.fit_transform(df.drop(['Label'],axis=1))

In [13]:
def ErrorClas(Y_lest, Y):
    """funcion que calcula el error de clasificación
    Y_lest: numpy array con la estimaciones de etiqueta
    Y: etiquetas reales
    retorna: error de clasificación (int)
    """
    error = 1 - np.sum(Y_lest == Y)/len(Y)
    
    return error

In [14]:
def KNN_Clasificacion(X_train, Y_train, X_test, k):
    """ Funcion que implementa el modelo de K-Vecino mas cercanos
        para clasificación
    X_train: es la matriz con las muestras de entrenamiento
    Y_train: es un vector con los valores de salida pra cada una de las muestras de entrenamiento
    X_test: es la matriz con las muestras de validación
    k (int): valor de vecinos a usar
    retorna: las estimaciones del modelo KNN para el conjunto X_test 
             esta matriz debe tener un shape de [row/muestras de X_test] 
             y las distancias de X_test respecto a X_train, estan matrix
             debe tener un shape de [rows de X_test, rows X_train]
             lo que es lo mismo [muestras de X_test, muestras de X_train]
    """
    if k > X_train.shape[0]:
        print("k no puede ser menor que las muestras de entrenamiento")
        return(None)
    distancias = scipy.spatial.distance_matrix(X_test, X_train) ##Calculo la distancia de las dos matrices... mismos elementos
    Yest = np.zeros(X_test.shape[0])
    
    for i in range (len(X_test)):
        index_vecinos = np.argsort(distancias[i])[:k]  #con argsot la ordeno con los vecinos más cercanos, me retorna el índice de los menores.
        k_vecinos = np.take(Y_train, index_vecinos)  ## Acá me traigo el valor de la matriz Yv, con el índice de los vecinos que obtuve arriba
        Yest[i] = stats.mode(k_vecinos)[0]   ##Como la función directamente me dice que es de clasificación, entonces hago solo la moda.

        
    
    return (Yest, distancias) 

In [15]:
#ejercicio de codigo
def train_test_split_fix(X,Y):
    """funcion que divide el conjunto de datos en
        entrenamiento y pruebas
        usando un proporcion fija de 30 %
        para el conjunto de pruebas.

    X: matriz de numpy con las muestras y caractersiticas
    Y: matriz de numpy con las las etiquetas reales
    retorna:
        Xtrain: conjunto de datos para entrenamiento
        Xtest: conjunto de datos para pruebas
        Ytrain: conjunto  de etiquetas para entrenamiento
        Ytest: conjunto de etiquetas para prueba 
    """
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y, train_size = 0.7, test_size = 0.3 )

    return (Xtrain, Xtest, Ytrain, Ytest)


In [16]:
from scipy.spatial import distance

In [17]:
def kernel_gaussiano(x):
    """Calcula el kernel gaussiano de x
    x: matriz/vector de numpy
    retorna: el valor de de kernel gaussiano
    """
    return np.exp((-0.5)*x**2)

def ParzenWindow(x,Data,h):
    """"ventana de parzen
    x: vector con representando una sola muestra
    Data: vector de muestras de entrenamiento
    h: ancho de la ventana de kernel
    retorna: el valor de ventana de parzen para una muestra
    """
    h = h
    Ns = Data.shape[0]
    suma = 0
    for k in range(Ns):
        u = distance.euclidean(x,Data[k,:])
        suma += kernel_gaussiano(u/h)
    return suma

In [18]:
#Ejercicio de código
def parzenClass(X_train, Y_train, X_test, h):
    """ Funcion que implementa metodo de ventana de parzen para
        para clasificación
    X_train: es la matriz con las muestras de entrenamiento
    Y_train: es un vector con los valores de salida pra cada una de las muestras de entrenamiento
    X_test: es la matriz con las muestras de validación
    h (float): ancho de h de la ventana
    retorna: - las estimaciones del modelo parzen para el conjunto X_test 
              esta matriz debe tener un shape de [row/muestras de X_test]
             - las probabilidades de la vetana [row/muestras de X_test, numero de clases]  
    """
        
    Yest = np.zeros(X_test.shape[0])
    clases = np.unique(Y_train)
    fds_matrix = np.zeros((X_test.shape[0], len(clases)))
    
    ## pista: recuerde el termino que acompaña al sumatoria (N)
    index = 0
    for n, sample in enumerate (X_test):
      max_prob = 0
      est_label = None
      j = 0
      for label in clases:
        indixes = np.argwhere(Y_train == label)
        train_elements = np.take(X_train, indixes.T[0], axis = 0)
        dim = train_elements.shape
        prob = ParzenWindow(sample,train_elements,h)/dim[0]
        fds_matrix[index][j] = prob
        if prob > max_prob:
          max_prob = prob
          est_label = label
        j += 1
      Yest[index] = est_label
      index +=1
    
    

    #Debe retornar un vector que contenga las predicciones para cada una de las muestras en X_val, en el mismo orden.  
    return Yest, fds_matrix

In [19]:
from sklearn.model_selection import StratifiedKFold


#ejercicio de codigo
def experimentarParzen (X, Y, hs):
    """Función que realiza los experimentos con knn usando
       una estrategia de validacion entrenamiento y pruebas
    X: matriz de numpy conjunto con muestras y caracteristicas
    Y: vector de numpy con los valores de las etiquetas
    ks: List[int/float] lista con los valores de k-vecinos a usar
    retorna: dataframe con los resultados, debe contener las siguientes columnas:
        - el ancho de ventana, el error medio de prueba, la desviacion estandar del error
    """
    
    
    # se usa la función para implementar la estrategia de validación.
    skf = StratifiedKFold(n_splits=4)
    resultados = pd.DataFrame()
    idx = 0
    # iteramos sobre los valores de hs
    for h in hs:
        # lista para almacenar los errores de cada iteración
        # de la validación
        error_temp = []
        
        for train, test in skf.split(X, Y):

            Xtrain = X[train,:]
            Ytrain = Y[train]
            Xtest = X[test,:]
            Ytest = Y[test]
            #normalizamos los datos
            scaler = StandardScaler()
            scaler.fit(Xtrain)
            Xtrain = scaler.transform(Xtrain)
            Xtest = scaler.transform(Xtest)
            
            Yest, probabilidades = parzenClass(Xtrain, Ytrain, Xtest, h)
            errorTest = ErrorClas(Yest, Ytest)
            error_temp.append(errorTest)
    
        resultados.loc[idx,'ancho de ventana'] = h 
        resultados.loc[idx,'error de prueba(media)'] = errorTest.mean()
        resultados.loc[idx,'error de prueba(desviación estandar)'] = errorTest.std()
        idx+=1
    return (resultados)

In [20]:
YEntrenamiento = np.array(YEntrenamiento)

In [21]:
YEntrenamiento = YEntrenamiento.astype(int)


In [22]:
print(YEntrenamiento)

[1 1 1 ... 0 0 0]


In [23]:
hs = [0.05, 0.1, 0.5, 1, 2, 5, 10]
experimentos_parzen = experimentarParzen(XEntrenamiento,YEntrenamiento, hs)
experimentos_parzen 


,ancho de ventana,error de prueba(media),error de prueba(desviación estandar)
0,0.05,0.148687,0.0
1,0.10,0.015557,0.0
2,0.50,0.001275,0.0
3,1.00,0.002550,0.0
4,2.00,0.039276,0.0
5,5.00,0.186177,0.0
6,10.00,0.262943,0.0
